In [1]:
import ROOT #Start by importing ROOT
import numpy as np
ROOT.enableJSVis() #need this to make interactive ROOT plots
c1 = ROOT.TCanvas() #declare the ROOT canvas
ROOT.gSystem.Load("/usr/local/software/quadis/build/lib/libquadis.so")

Welcome to JupyROOT 6.22/02


0

In [2]:
infile = ROOT.TFile("/home/vasu/quadis/build/bin/60kev-right/data33.root")
tree = infile.Get("T2")
num_files = tree.GetEntries()
print (num_files)

45000


In [3]:
infile2 = ROOT.TFile("/home/vasu/PPC/PPC_SigGen/SignalLib_root/new_OutputRZperfect.root")
libtree = infile2.Get("T1")
nevent = ROOT.NEvent()
libtree.SetBranchAddress("NEvent",nevent)

rtrue = np.zeros(num_files)
ztrue = np.zeros(num_files)
ravg = np.zeros(num_files)
zavg = np.zeros(num_files)
R_new = np.zeros(num_files)
Z_new = np.zeros(num_files)
rbest = np.zeros(10)
zbest = np.zeros(10)
ind = 0

vector_chi = ROOT.std.vector('vector<double>')()
tree.SetBranchAddress("blrPulse_Chis", vector_chi)
vector_R = ROOT.std.vector('double')()
tree.SetBranchAddress("R", vector_R)
vector_Z = ROOT.std.vector('double')()
tree.SetBranchAddress("Z", vector_Z)
for i in range (num_files):
    tree.GetEntry(i)
    rtrue[i] = vector_R[0]
    ztrue[i] = vector_Z[0]
    ordered_index = np.argsort(vector_chi[0])
    for k in range (10):
        libtree.GetEntry(ordered_index[k])
        rbest[k] = nevent.GetPulseData(1).GetParameterByName("R")
        zbest[k] = nevent.GetPulseData(1).GetParameterByName("Z")
    ravg[i] = np.mean(rbest)
    zavg[i] = np.mean(zbest)
    R_new[i] = (rtrue[i]-ravg[i])
    Z_new[i] = (ztrue[i]-zavg[i])
    

In [4]:
Rrms = []
Zrms = []
for i in range (num_files):
    if ((i%1000)==0):
        hr = ROOT.TH1D("hr","hr",67,-33,34)
        hz = ROOT.TH1D("hz","hz",103,-51,52)
        for j in range (1000):
            hr.Fill(R_new[i+j])
            hz.Fill(Z_new[i+j])
        Rrms.append(hr.GetStdDev())
        Zrms.append(hz.GetStdDev())
        hr.Reset()
        hz.Reset()

Warning in <TFile::Append>: Replacing existing TH1: hr (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: hz (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: hr (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: hz (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: hr (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: hz (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: hr (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: hz (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: hr (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: hz (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: hr (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: hz (Potential memory leak).
Warning in <TFile::Append>: Replacing ex

In [5]:
R0 = np.zeros(int(num_files/1000), dtype = 'int')
Z0 = np.zeros(int(num_files/1000), dtype = 'int')

for i in range (len(rtrue)):
    if ((i%1000)==0):
        val = int(i/1000)
        R0[val] = rtrue[i]
        Z0[val] = ztrue[i]

In [6]:
f1 = open('/home/vasu/PPC/PPC_SigGen/arrays/60kev-right/data33','w')
for i in range (len(Rrms)):
    f1.write(str(R0[i]))
    f1.write('\t')
    f1.write(str(Rrms[i]))
    f1.write('\t')
    f1.write(str(Z0[i]))
    f1.write('\t')
    f1.write(str(Zrms[i]))
    f1.write('\n')
f1.close()